<a href="https://colab.research.google.com/github/HariniMohan97/Assignment7/blob/master/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, SeparableConv2D, GlobalAveragePooling2D, Add
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 10
num_filter = 20

# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


#cuts input (space) into 2 blocks and places on top of another (depth)
#helps concatenating two layers with diff dimensions
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2) 

  


Aim: To build the network shown below:-

![alt text](https://2.bp.blogspot.com/-HQo2Kx39Q6A/WsqZQn5pJHI/AAAAAAAAVhY/FBkgOeS06vQNvZN2KI2hhGZmfS1cNPg8wCLcBGAs/s1600/enasdiscoverednetwork.png)

##-->Functional api menthod used. 
##-->Space to Depth introduced here which helps modify dimensions of our output channel that inturn helps to concatenate with other output channels from other layers.

Application:- 

Everytime we want to concatenate an output from a layer that comes before a max pool layer(outputA), along with an output from a layer that comes after the maxpool layer(outputB), we need to use space to depth. This operation is performed on top of outputA to match its dimensions with outputB since the dimensions of outputB is < that out outputA

##-->Padding = 'same' also helps in maintaining the dimensions

##-->Adding outputs of two layers helps integrate receptive fields.

In [0]:
input = Input(shape=(img_height, img_width, channel,))

#32 (same) > 64 (same) > 128 (same) > 256 (same) > 512 (same) > MP


Layer1 = SeparableConv2D(8, (5,5),padding = 'same', strides = (1,1), depth_multiplier = 1)(input) #5
Layer1=BatchNormalization()(Layer1)
Layer1=Activation('relu')(Layer1)
Layer1=Dropout(0.25)(Layer1)

Layer2 = Conv2D(8, (5,5), strides=(1,1), padding='same', use_bias=False)(Layer1) #9
Layer2=BatchNormalization()(Layer2)
Layer2=Activation('relu')(Layer2)
Layer2=Dropout(0.25)(Layer2)

Layer3 = Conv2D(8, (5,5), strides=(1,1), padding='same', use_bias=False)(Layer2)#13
Layer3=BatchNormalization()(Layer3)
Layer3=Activation('relu')(Layer3)
Layer3=Dropout(0.25)(Layer3)

LayerAdd1 = concatenate([Layer3,Layer1])

Layer4 = SeparableConv2D(8, (5,5),padding = 'same', strides = (1,1), activation='relu', depth_multiplier = 1)(LayerAdd1)

LayerAdd2 = concatenate([Layer1,Layer4])

Layer5 = MaxPooling2D(pool_size=(2, 2))(LayerAdd2)

Layer6 = SeparableConv2D(10, (3,3),padding = 'same', strides = (1,1), depth_multiplier = 1)(Layer5)
Layer6=BatchNormalization()(Layer6)
Layer6=Activation('relu')(Layer6)
Layer6=Dropout(0.25)(Layer6)

Layer4 = Lambda(space_to_depth_x2)(Layer4)
Layer3 = Lambda(space_to_depth_x2)(Layer3)
Layer1 = Lambda(space_to_depth_x2)(Layer1)
Layer2 = Lambda(space_to_depth_x2)(Layer2)

LayerAdd3 = concatenate([Layer4,Layer6])

Layer7 = Conv2D(10, (5,5), strides=(1,1), padding='same', use_bias=False)(LayerAdd3)
Layer7=BatchNormalization()(Layer7)
Layer7=Activation('relu')(Layer7)
Layer7=Dropout(0.25)(Layer7)

LayerAdd4 = concatenate([Layer3,Layer7,Layer6,Layer4])

Layer8 = SeparableConv2D(10, (3,3),padding = 'same', strides = (1,1), depth_multiplier = 1)(LayerAdd4)
Layer8=BatchNormalization()(Layer8)
Layer8=Activation('relu')(Layer8)
Layer8=Dropout(0.25)(Layer8)

LayerAdd5 = concatenate([Layer8,Layer7,Layer3,Layer6,Layer1,Layer4])

Layer9 = SeparableConv2D(10, (5,5),padding = 'same', strides = (1,1), activation='relu', depth_multiplier = 1)(LayerAdd5)

LayerAdd6 = concatenate([Layer8,Layer9,Layer6,Layer1,Layer4])

Layer10 = MaxPooling2D(pool_size=(2, 2))(LayerAdd6)

Layer7 = Lambda(space_to_depth_x2)(Layer7)
LayerAdd7 = concatenate([Layer7,Layer10])

Layer11 = Conv2D(16, (5,5), strides=(1,1), padding='same', use_bias=False)(LayerAdd7)
Layer11=BatchNormalization()(Layer11)
Layer11=Activation('relu')(Layer11)
Layer11=Dropout(0.25)(Layer11)


Layer1 = Lambda(space_to_depth_x2)(Layer1)
Layer8 = Lambda(space_to_depth_x2)(Layer8)
Layer2 = Lambda(space_to_depth_x2)(Layer2)
LayerAdd8 = concatenate([Layer11,Layer1,Layer8,Layer2])

Layer12 = SeparableConv2D(16, (5,5),padding = 'same', strides = (1,1), depth_multiplier = 1)(LayerAdd8)
Layer12=BatchNormalization()(Layer12)
Layer12=Activation('relu')(Layer12)
Layer12=Dropout(0.25)(Layer12)

Layer3 = Lambda(space_to_depth_x2)(Layer3)
Layer6 = Lambda(space_to_depth_x2)(Layer6)
LayerAdd9 = concatenate([Layer11,Layer12,Layer3,Layer2,Layer6])

Layer13 = Conv2D(16, (3,3), strides=(1,1), padding='same', use_bias=False)(LayerAdd9)
Layer13=BatchNormalization()(Layer13)
Layer13=Activation('relu')(Layer13)
Layer13=Dropout(0.25)(Layer13)

Layer4 = Lambda(space_to_depth_x2)(Layer4)
LayerAdd10 = concatenate([Layer13,Layer8,Layer3,Layer12,Layer1,Layer4,Layer6])

Layer14 = SeparableConv2D(16, (5,5),padding = 'same', strides = (1,1), activation='relu', depth_multiplier = 1)(LayerAdd10)

LayerAdd11 = concatenate([Layer12,Layer14,Layer8])

Layer15 = Activation('softmax') (LayerAdd11)

Layer16 = Conv2D(10, (1,1), strides=(1,1), padding='same', use_bias=False)(Layer15) #1x1 before GAP. to get 10 output channels finally

Layer17 = GlobalAveragePooling2D()(Layer16) #GAP on the 10 obtained channels  #acts just like FCL(fully connected layer) 

output = (Layer17)



In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d_173 (Separable (None, 32, 32, 8)    107         input_32[0][0]                   
__________________________________________________________________________________________________
batch_normalization_135 (BatchN (None, 32, 32, 8)    32          separable_conv2d_173[0][0]       
__________________________________________________________________________________________________
activation_152 (Activation)     (None, 32, 32, 8)    0           batch_normalization_135[0][0]    
___________________________________________________________________________________________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

## Accuracy, loss stuck in one place. Still unsure why it happens.
## And the model takes forever to train! :'(
## What could be done to improve the acuracy and reduce the time taken to train?
## Is it because the network is too complex with too many params?

In [0]:
model.fit(x_train, y_train,
                    batch_size=10,
                    epochs=100,
                    verbose=1,
                    validation_data=(x_test, y_test)) 


Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 1027s 21ms/step - loss: 8.0343 - acc: 0.1038 - val_loss: 8.0000 - val_acc: 0.1151
Epoch 2/100
50000/50000 [==============================] - 996s 20ms/step - loss: 8.2456 - acc: 0.1003 - val_loss: 9.6709 - val_acc: 0.1000
Epoch 3/100
50000/50000 [==============================] - 1018s 20ms/step - loss: 9.6709 - acc: 0.1000 - val_loss: 9.6709 - val_acc: 0.1000
Epoch 4/100
50000/50000 [==============================] - 973s 19ms/step - loss: 9.6709 - acc: 0.1000 - val_loss: 9.6709 - val_acc: 0.1000
Epoch 5/100
50000/50000 [==============================] - 959s 19ms/step - loss: 9.6709 - acc: 0.1000 - val_loss: 9.6709 - val_acc: 0.1000
Epoch 6/100
50000/50000 [==============================] - 973s 19ms/step - loss: 9.6709 - acc: 0.1000 - val_loss: 9.6709 - val_acc: 0.1000
Epoch 7/100
50000/50000 [==============================] - 966s 19ms/step - loss: 9.6709 - acc: 0.1000 - val

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

files.download('Yolo_Basic_model.h5')